In [1]:
# import necessary packages
import pandas as pd
import numpy as np
import nltk

In [2]:
# synonym_dict
synonym_dict = {
    "đôi": "cặp",
    "couple": "cặp",
    "mũ": "nón",
    "gile": "ghile",
    "ghi lê": "ghile",
    "triệt": "nhổ",
    "tựa": "dựa",
    "nến": "đèn cầy",
    "bó": "ép",
    "dây nịt": "thắt lưng",
    "gôn": "golf",
    "sang": "to",
    "adapter": "cáp chuyển",
    "bao tay": "găng tay",
    "khử": "trừ",
    "khăn choàng": "khăn quàng",
    "cốc": "ly",
    "thái thịt": "cắt thịt",
    "bào đá": "xay đá",
    "áo lưới": "áo ren",
    "xà phòng": "xà bông",
    "bạt phủ": "áo phủ",
    "áo trùm": "áo phủ",
    "bạt trùm": "áo phủ",
    "splitter": "bộ chia",
    "nắp lưng": "vỏ lưng",
    "phích cắm": "ổ cắm",
    "cafe": "cà phê",
    "ôtô": "xe hơi",
    "ô tô": "xe hơi",
    "oto": "xe hơi",
    "drap": "ga",
    "case": "ốp lưng",
    "balo": "túi xách",
    "ba lô": "túi xách",
    "balô": "túi xách",
    "cặp xách": "túi xách",
    "headphone": "tai nghe",
    "gold": "vàng",
    "xoè": "xòe",
    "massage": "mát xa",
    "tay ngắn": "ngắn tay",
    "tay dài": "dài tay",
    "váy": "đầm",
    "sạc nhanh": "fast charge",
    "bóp": "ví",
    "áo ngực": "áo lót",
    "hdd cable": "cáp ổ cứng",
    "makbook": "macbook",
    "mabook": "macbook",
    "fan": "quạt",
    "chè": "trà",
#     "ô": "dù",
    "masage": "mát xa",
    " i ": " ",
    "6 plus": "6+",
    "7 plus": "7+",
    " pro": "_pro",
    "iphone 6": "iphone_6",
    "iphone 6 plus": "iphone_6+",
    "iphone 7": "iphone_7",
    "iphone 7 plus": "iphone_7+",
    "iphone 7plus": "iphone_7+",
    "iphone x": "iphone 10",
    " ii": " 2",
    "thái": "cắt",
    "sạc": "nạp",
    "i pad": "ipad"
}

special_character = ['|', ',', ':', '\'', '"', '(', ')', '{', '}', '[',']', '_', '-']

In [3]:
# necessary functions
def remove_special_character(sentence):
    for c in special_character:
        sentence = sentence.replace(c," ")
    return sentence

def replace_synonym(sentence):
    sentence = sentence.lower()
    for k,v in synonym_dict.items():
        sentence = sentence.replace(k,v)
    return sentence

def check_duplicate(word_list,n):
    ngrams = list(nltk.ngrams(word_list,n))
    freq = []
    for ngram in set(ngrams):
        sentence = " ".join(word_list)
        substring = " ".join(ngram)
        count = sentence.count(substring)
        freq.append((ngram,count))
        
    freq = sorted(freq, key = lambda x: x[1])
    return freq

def max_ngrams(sentence):
    # create a word list
    word_list = nltk.word_tokenize(sentence)
    
    # binary search for max ngrams
    max_len = int(len(word_list)/2)
    min_len = 1
    
    result_word = None
    while 1:
        current_len = int((min_len + max_len)/2)
        [my_word, num_appear] = check_duplicate(word_list, current_len).pop()
        if num_appear > 1: # has repeated
            [result_word, result_appear] = [my_word, num_appear]
            min_len = current_len + 1
            if min_len > max_len:
                break
        else: # no repeat
            max_len = current_len - 1
            if max_len < min_len:
                break
    
    [result_word_1, result_appear_1] = check_duplicate(word_list,min_len).pop()
    if result_word == None:
        return [result_word_1,result_appear_1]
    else:
        if result_appear_1 > result_appear:
            return [result_word_1,result_appear_1]
        else:
            return [result_word,result_appear]
    
def remove_repeat_word(sentence):
    word_list = nltk.word_tokenize(sentence)
    my_dict = {}
    original_len = len(word_list)
    for word in word_list:
        if my_dict.get(word) == None:
            my_dict[word] = 1
        else:
            my_dict[word] += 1
            
    count = 0
    for word in word_list:
        if my_dict[word] == 1:
            count += 1
            
    ratio = count/original_len
    return [original_len, count, ratio]

def min_word_distance(sentence):
    [result_word, result_appear] = max_ngrams(sentence)
    substring = " ".join(result_word)
    
    # sentence without substring
    if sentence.count(substring) < 2:
        return (len(list(nltk.word_tokenize(sentence))),1,0,0)
    
    my_sentence = sentence.replace(substring,'')
    len_word = len(list(nltk.word_tokenize(my_sentence)))
    #len_word = len(list(nltk.word_tokenize(sentence)))
        
    substring_list = sentence.split(substring)
    
    if substring_list[0] == '':
        isBegin = 1
    else:
        isBegin = 0
        
    if substring_list[-1] == '':
        isEnd = 1
    else:
        isEnd = 0
        
    if (len(substring_list[0]) == 0) and (len(substring_list[-1]) == 0):
        return (0,0,isBegin,isEnd)
    
    result = []
    for i,value in enumerate(substring_list):
        tmp = list(nltk.word_tokenize(value))
        if (i == 0) or (i == len(substring_list)-1):
            pass
        else:
            result.append((value,len(tmp)))
            
    result = sorted(result, key = lambda x: x[1])
    
    if len_word == 0:
        len_word = 1;
        
    return (result[0][1],result[0][1]/len_word,isBegin,isEnd)

def convert_to_non_diacritics(title):
    my_dict = {
        # character a
        'à': 'a', 'ả': 'a', 'ã': 'a', 'á': 'a', 'ạ': 'a',
        'ă': 'a', 'ằ': 'a', 'ẳ': 'a', 'ẵ': 'a', 'ắ': 'a',
        'ặ': 'a', 'â': 'a', 'ầ': 'a', 'ẩ': 'a', 'ẫ': 'a',
        'ấ': 'a', 'ậ': 'a',
        
        # character e
        'è': 'e', 'ẻ': 'e', 'ẽ': 'e', 'é': 'e', 'ẹ': 'e',
        'ê': 'e', 'ề': 'e', 'ể': 'e', 'ễ': 'e', 'ế': 'e',
        'ệ': 'e',
        
        # character i
        'ì': 'i', 'ỉ': 'i', 'ĩ': 'i', 'í': 'i', 'ị': 'i',
        
        # character o
        'ò': 'o', 'ỏ': 'o', 'õ': 'o', 'ó': 'o', 'ọ': 'o',
        'ơ': 'o', 'ờ': 'o', 'ở': 'o', 'ỡ': 'o', 'ớ': 'o',
        'ợ': 'o', 'ô': 'o', 'ồ': 'o', 'ổ': 'o', 'ỗ': 'o',
        'ố': 'o', 'ộ': 'o',
        
        # character u
        'ù': 'u', 'ủ': 'u', 'ũ': 'u', 'ú': 'u', 'ụ': 'u',
        'ư': 'u', 'ừ': 'u', 'ử': 'u', 'ữ': 'u', 'ứ': 'u',
        'ự': 'u',
        
        # character y
        'ỳ': 'y', 'ỷ': 'y', 'ỹ': 'y', 'ý': 'y', 'ỵ': 'y',
        
        # character d
        'đ': 'd'
    }
    
    s = ""
    for c in title:
        if my_dict.get(c) != None:
            s += my_dict[c]
        else:
            s += c
    return s

def choose_title(title):
    title = replace_synonym(title)
    title_non_diacritics = convert_to_non_diacritics(title)
    if ((len(max_ngrams(title)[0]) + 2 <= len(max_ngrams(title_non_diacritics)[0])) or
        ((len(max_ngrams(title_non_diacritics)[0]) >= 3) 
             and (len(max_ngrams(title_non_diacritics)[0]) > len(max_ngrams(title)[0])))):
        return title_non_diacritics
    return title

def is_valid(s1,s2):
    word_list1 = s1.split(" ")
    word_list2 = s2.split(" ")
    
    if (s2.find(s1) != -1) or (s1.find(s2) != -1):
        return False
    
    if len(word_list1) <= len(word_list2):
        list1 = word_list1.copy()
        list2 = word_list2.copy()
    else:
        list1 = word_list2.copy()
        list2 = word_list1.copy()
        
    curr_idx = 0
    count = 0
    for word1 in list1:
        flag = True
        for i,word2 in enumerate(list2[curr_idx:],start = curr_idx):
            if word1 == word2:
                curr_idx = i + 1
                flag = False
                break
        if flag:
            return False
        
    return True

def my_tokenize(s,n):
    word_list = list(s.split(" "))
    result = []
    for i in range(len(word_list) - n + 1):
        result.append(" ".join(word_list[i:i+n]))
        
    return result

def check_skipgram(s,n):
    sentence_list = my_tokenize(s,n)
    curr_idx = 0
    for sentence in sentence_list:        
        idx = s[curr_idx:].find(sentence)
        part1 = s[:max(0,curr_idx+idx-1)]
        part2 = s[curr_idx+idx+len(sentence)+1:]
        part1_tokenize = my_tokenize(part1,n+1)
        part2_tokenize = my_tokenize(part2,n+1)
        for token in part1_tokenize:
            if is_valid(sentence,token):
                return [sentence,token]
        for token in part2_tokenize:
            if is_valid(sentence,token):
                return [sentence,token]
            
        curr_idx += idx

    return [None,None]

def check_skipgram_overall(s):
    word_list = s.split(" ")
    n = int(len(word_list)/2)
    tmp = [None,None]
    while n >= 4:
        tmp = check_skipgram(s,n)
        if tmp[0] != None:
            break
        else:
            n -= 1
            
    len_original_s = len(max_ngrams(s)[0])
    if tmp == [None,None]:
        len_skip_s = 0
#         return [None,None,0]
    else:
        len_skip_s = len(list(tmp[1].split(" ")))
#         return [tmp[0],tmp[1],len(list(tmp[1].split(" ")))]

    if len_skip_s >= max(len_original_s,4):
        return s.replace(tmp[0],tmp[1])
    else:
        return s
    
def preprocess(s):
    s = remove_special_character(s)
    s = s.lower()
    
    # remove multiple space
    result = ''
    isStart = 1
    isSpace = 0
    for c in s:
        if c != ' ':
            result += c
            isSpace = 0
        else: # c == ' '
            if isStart == 1:
                isStart = 0
                pass
            else:
                if isSpace == 0:
                    result += c
                    isSpace = 1
                else:
                    pass
                
    if result[-1] == ' ':
        return result[:-1]
    else:
        return result

In [4]:
# create features for title
def create_feature(df_original_title):
    # preprocess title
    df_title = df_original_title.apply(lambda x: preprocess(x))
    df_title = df_title.apply(lambda x: choose_title(x))
    df_title = df_title.apply(lambda x: check_skipgram_overall(x))
    
    ngrams = df_title.apply(lambda x: max_ngrams(x))
    non_repetition = df_title.apply(lambda x: remove_repeat_word(x))
    min_distance = df_title.apply(lambda x: min_word_distance(x))
    skipgram = df_title.apply(lambda x: check_skipgram_overall(x))
    
    # feature engineering step
    feature_01 = ngrams.apply(lambda x: x[0])
    feature_02 = ngrams.apply(lambda x: len(x[0]))
    feature_03 = ngrams.apply(lambda x: x[1])
    feature_04 = non_repetition.apply(lambda x: x[0])
    feature_05 = non_repetition.apply(lambda x: x[1])
    feature_06 = non_repetition.apply(lambda x: x[2])
    feature_07 = min_distance.apply(lambda x: x[0])
    feature_08 = min_distance.apply(lambda x: x[1])
    feature_09 = min_distance.apply(lambda x: x[2])
    feature_10 = min_distance.apply(lambda x: x[3])
    
    df_feature = pd.concat([
        df_original_title,
        feature_01,
        feature_02,
        feature_03,
        feature_04,
        feature_05,
        feature_06,
        feature_07,
        feature_08,
        feature_09,
        feature_10
    ], axis=1)
    
    df_feature.columns = ['title',
                         'max_ngrams',
                         'len_max_ngrams',
                         'num_repetition_max_ngrams',
                         'len_original_words',
                         'len_non_repetition_words',
                         'meaning_ratio',
                         'min_distance',
                         'relative_dist_ratio',
                         'isBegin',
                         'isEnd']
    
    return df_feature

In [5]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import svm
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# learn random forest model
def learn_rf(train_X, train_y):
    param_grid = [
        {
            'n_estimators': [200, 300, 400, 500, 1000, 2000],
            'max_features': [2 ,3, 4, 5, 6, 7, 8, 9],
            'bootstrap': [True, False],
            'min_samples_split': [5, 10, 20, 50, 100]
        }
    ]
    
    forest_classification = RandomForestClassifier()
    
    # train across 5 folds
    rf_grid_search = GridSearchCV(forest_classification,
                                 param_grid,
                                 cv = 5,
                                 n_jobs = -1)
    
    rf_grid_search.fit(train_X, train_y)
    rf_best_model = rf_grid_search.best_estimator_
    
    return rf_best_model

def calculate_prediction_prob_rf(rf_best_model,titles):
    df_feature = create_feature(titles)
    
    # extract necessary data only
    df_shortlisted = df_feature[['len_max_ngrams',
                                'num_repetition_max_ngrams',
                                'len_original_words',
                                'len_non_repetition_words',
                                'meaning_ratio',
                                'min_distance',
                                'relative_dist_ratio',
                                'isBegin',
                                'isEnd']]
    
    prediction_prob_rf = rf_best_model.predict_proba(df_shortlisted)[:,1]
    
    return prediction_prob_rf

In [7]:
# create train_X, train_y
df = pd.read_csv('labeled_data_05.csv', header = None, encoding = 'utf-8')
df.columns = ['id', 'title', 'class']
df_feature = create_feature(df['title'])

train_X = df_feature[['len_max_ngrams',
                     'num_repetition_max_ngrams',
                     'len_original_words',
                     'len_non_repetition_words',
                     'meaning_ratio',
                     'min_distance',
                     'relative_dist_ratio',
                     'isBegin',
                     'isEnd']]

train_y = df['class']

rf_best_model = learn_rf(train_X, train_y)

In [8]:
# Load data
df_b_0_search_result = pd.read_csv('b_0_search_result.txt', sep = '\n', header = None, names = ['titles'])
df_default_search_result = pd.read_csv('default_search_result.txt', sep = '\n', header = None, names = ['titles'])

# calculate prediction prob for titles
predict_prob_b_0 = calculate_prediction_prob_rf(rf_best_model,df_b_0_search_result['titles'])
predict_prob_default = calculate_prediction_prob_rf(rf_best_model,df_default_search_result['titles'])

# output result
b_0_search_result_predict_prob = pd.concat([df_b_0_search_result, pd.Series(predict_prob_b_0)], axis = 1)
default_search_result_predict_prob = pd.concat([df_default_search_result, pd.Series(predict_prob_default)], axis = 1)

# save file
b_0_search_result_predict_prob.to_csv('b_0_predict_prob', sep='\t', index = False)
default_search_result_predict_prob.to_csv('default_predict_prob', sep='\t', index = False)